## DAUM 뉴스 URL, 제목 크롤링

github에서 소스코드 다운받는법

row -> url복사
ctrl + r -> cmd
cd 다운받을 위치 -> Enter -> wget위치 + 복사한url 

In [35]:
import requests
import lxml.html
import pandas as pd
import sqlite3
from pandas.io import sql
import os

In [36]:
pd.date_range('2016-01-01', '2017-01-01')

DatetimeIndex(['2016-01-01', '2016-01-02', '2016-01-03', '2016-01-04',
               '2016-01-05', '2016-01-06', '2016-01-07', '2016-01-08',
               '2016-01-09', '2016-01-10',
               ...
               '2016-12-23', '2016-12-24', '2016-12-25', '2016-12-26',
               '2016-12-27', '2016-12-28', '2016-12-29', '2016-12-30',
               '2016-12-31', '2017-01-01'],
              dtype='datetime64[ns]', length=367, freq='D')

함수만들기

In [37]:
# # 상세페이지 & 특수문자제거
# import re
# import string 
# # url = 'https://news.v.daum.net/v/20200505000102404'
# # 여러 string을 한개의 string으로 합치기
# def get_detail(url):    
#     body = []
#     response = requests.get(url)
#     root = lxml.html.fromstring(response.content)
#     for p in root.xpath('//*[@id="harmonyContainer"]/section/p'):
#         body.append(p.text)
#     full_body=' '.join(body)
#     # print(type(full_body))
#     # print(string.punctuation)      특수문자
#     # 특수문자 제거
#     # punc = '[!\"\“#$%&\'()*+,-./:;<=>?[\]^_`{|}~·]'
#     return re.sub('[!\"\“#$%&\'()*+,-./:;<=>?[\]^_`{|}~·‘’]','', full_body)    #[]: 여기중 한개

import re
import string
# url = 'https://news.v.daum.net/v/20200505000102404'

def get_detail(url):
    body = []
    punc = '[!"“”‘’#$%&\'()*+,-/:;<=>?[\\]^_`{|}~·]'  #string.punctuation & 필요한 특수문자 추가
    response = requests.get(url)
    root = lxml.html.fromstring(response.content)
    for p in root.xpath('//*[@id="harmonyContainer"]/section/p'): 
        if p.text:
            body.append(re.sub(punc, '' , p.text)) # 정리 후 특수문자 제거
    full_body = ' '.join(body)    # list 하나로 조인
    
    return full_body
get_detail('https://news.v.daum.net/v/20200505000102404')

'LG전자 스마트TV 리모컨 클릭 한 번에 왓챠플레이가 연결된다 왓챠와 LG전자는 LG 2019년형 스마트TV부터 스마트TV 리모컨에서 영화 버튼을 클릭하면 왓챠플레이 서비스가 바로 연동되도록 지원한다 왓챠플레이를 구독하는 고객 편의성을 강화했다 LG 신형 스마트TV를 보유한 왓챠플레이 고객은 편리하게 서비스를 이용할 수 있게 됐다 기존에도 삼성전자LG전자 스마트TV 메뉴와 유료방송 셋톱박스 등을 통해 왓챠플레이를 시청할 수 있었다 왓챠플레이 애플리케이션앱을 찾아야 하는 번거로움이 있었다 박태훈 왓챠 대표는 국내외 제조사는 물론 통신사 방송사와 서비스 협업을 논의 중이라며 왓챠플레이를 더 쉽게 접할 수 있도록 계속 협력해나갈 것이라고 말했다 왓챠플레이뿐만 아니라 웨이브 넷플릭스 디즈니플러스디즈니 등 대다수 온라인동영상서비스OTT가 스마트TV유료방송과 고객 접점을 확대하는 추세다 스마트폰태블릿PC 등 모바일은 물론 TV와 PC 등 다양한 디바이스에서 클릭 한 번에 서비스를 이용하도록 지원 접근성을 강화했다 고객 접점을 확대해 가입 유인을 늘리기 위한 전략이다 넷플릭스는 LG유플러스와 LG헬로비전 딜라이브 등 IPTV케이블TV와 서비스를 연동 쉽게 볼 수 있도록 지원한다 삼성LG 스마트TV 앱 또는 리모컨에서 넷플릭스NETFLIX 버튼 클릭 한 번에 서비스를 이용할 수 있도록 지원한다 웨이브와 CJ ENM 티빙TVING 등 국산 OTT는 물론 미국유럽 등 해외에서 서비스를 제공 중인 디즈니 역시 삼성LG전자 스마트TV에서 서비스를 제공한다 유료방송 플랫폼 차원에서 OTT와 제휴도 적극적이다 KT스카이라이프는 통합 OTT 서비스 토핑 출시 1주년을 맞아 국내 최대 애니메이션 스트리밍 플랫폼 라프텔 서비스를 추가했다 이달 초부터 라프텔을 통해 진격의 거인 소드 아트 온라인 도쿄 구울 나루토 원피스 등 인기 애니메이션을 회차별 결제 없이 월 9900원에 볼 수 있다 제공 OTT를 지속 확대할 계획이다 왓챠플레이와 웨이브는 토핑에서 유료 가입시청할 수 있다 유튜브는 무료

In [38]:
#DB에 저장하기
def db_save(NEWS_LIST):
    with sqlite3.connect(os.path.join('.','sqliteDB')) as con: # sqlite DB 파일이 존재하지 않는 경우 파일생성
        try:
            NEWS_LIST.to_sql(name = 'NEWS_LIST', con = con, index = False, if_exists='append') 
            #if_exists : {'fail', 'replace', 'append'} default : fail
        except Exception as e:
            print(str(e))
        print(len(NEWS_LIST), '건 저장완료..')

In [39]:
# 방법1 selector 사용하기
response = requests.get('https://news.daum.net/breakingnews/digital?regDate=20200505')
root = lxml.html.fromstring(response.content)
for a in root.cssselect('a'):
    url = a.get('href')
    print(url, a.text)

#kakaoBody 본문 바로가기
#kakaoGnb 메뉴 바로가기
https://news.daum.net/ None
https://entertain.daum.net None
https://sports.media.daum.net/sports None
/ None
/society None
/politics None
/economic None
/foreign None
/culture None
/digital None
/ranking/popular None
/series/ None
/photo None
/tv/ None
/1boon None
https://gallery.v.daum.net/p/home None
http://search.daum.net/search?w=tot&DA=23W&rtmaxcoll=Z8T&q=서울특별시날씨 
										
http://search.daum.net/search?w=tot&DA=23W&rtmaxcoll=Z8T&q=수원시 권선구날씨 
										
http://search.daum.net/search?w=tot&DA=23W&rtmaxcoll=Z8T&q=인천광역시날씨 
										
http://search.daum.net/search?w=tot&DA=23W&rtmaxcoll=Z8T&q=대구광역시날씨 
										
http://search.daum.net/search?w=tot&DA=23W&rtmaxcoll=Z8T&q=대전광역시날씨 
										
http://search.daum.net/search?w=tot&DA=23W&rtmaxcoll=Z8T&q=광주광역시날씨 
										
http://search.daum.net/search?w=tot&DA=23W&rtmaxcoll=Z8T&q=부산광역시날씨 
										
http://search.daum.net/search?w=tot&DA=23W&rtmaxcoll=Z8T&q=울산광역시날씨 
										
http://search.daum.net/se

In [40]:
# 방법1 xpath 사용하기
page = 1  # 시작 페이지
max_page = 0
dt = '20200504'   #날짜

while(True):
    df_list = []
    response = requests.get('https://news.daum.net/breakingnews/digital?page={}&regDate={}'\
                            .format(page, dt))
    root = lxml.html.fromstring(response.content)
    for li in root.xpath('//*[@id="mArticle"]/div[3]/ul/li'):
            a = li.xpath('div/strong/a')[0]
            url = a.get('href')
            df_list.append(
                pd.DataFrame({
                'URL' : [url],
                'TITLE' : [a.text],
                'ARTICLS' : [get_detail(url)]
            })
            )
    df_10 = pd.concat(df_list)
    
    db_save(df_10)
    
# 페이지중 가장큰 숫자  
    for a in root.xpath('//*[@id="mArticle"]/div[3]/div/span/a'):
    #에러처리    
        try:
            num = int(a.text)     # 숫자로 변환
            if max_page < num:
                max_page = num
        except:  #예외처리
            pass
    
    # 다음페이지로 이동하는 버튼이 있는지 없는지    
    span = root.xpath('//*[@id="mArticle"]/div[3]/div/span/a[11]/span/a[@class="btn_page btn_page"]')
    #print(len(span)) # 속성유무 없으면 0 있으면 1
    
    if (len(span) <= 0) & (page >= max_page):
        break;
    else:
        page = page + 1

15 건 저장완료..
15 건 저장완료..
15 건 저장완료..
15 건 저장완료..
15 건 저장완료..
15 건 저장완료..
15 건 저장완료..
15 건 저장완료..
15 건 저장완료..
15 건 저장완료..


In [22]:
# db 내용 삭제
def db_delete():
    with sqlite3.connect(os.path.join('.','sqliteDB')) as con: # sqlite DB 파일이 존재하지 않는 경우 파일생성
        try:
            cur = con.cursor()
            sql = 'DELETE FROM NEWS_LIST'
            cur.execute(sql)
        except Exception as e:
            print(str(e))

In [41]:
def db_select():
    with sqlite3.connect(os.path.join('.','sqliteDB')) as con: # sqlite DB 파일이 존재하지 않는 경우 파일생성
        try:
            query = 'SELECT * FROM NEWS_LIST'
            NEWS_LIST = pd.read_sql(query, con = con)
        except Exception as e:
            print(str(e)) 
        return NEWS_LIST

In [42]:
print(db_select())

                                        URL  \
0    https://v.daum.net/v/20200504222054332   
1    https://v.daum.net/v/20200504213822895   
2    https://v.daum.net/v/20200504213801893   
3    https://v.daum.net/v/20200504211329597   
4    https://v.daum.net/v/20200504210842530   
..                                      ...   
295  https://v.daum.net/v/20200504151946902   
296  https://v.daum.net/v/20200504151714833   
297  https://v.daum.net/v/20200504151615795   
298  https://v.daum.net/v/20200504151610793   
299  https://v.daum.net/v/20200504151340709   

                                             TITLE  \
0               애플, 13인치 맥북프로 출시.."10세대 인텔 CPU 탑재"   
1                   코로나가 만든 '조용한 바다'..고래들의 수다 시작될까   
2    편리한 전기 생활 위한 숨은 카드 '주파수' [찌릿찌릿(知it智it) 전기 교실]   
3               국내 IT업계 "정부·국회, 전기통신사업법 개정논의 중단하라"   
4           '14일' 지난 코로나19 확진자 동선..방통위, 포털 등 삭제 요청   
..                                             ...   
295               '줌' 영상통화 하면 계좌 털린다? 로그인 기록 거래 왕성   
296

# db를 Excel로 저장하기

In [43]:
NEWS_LIST = db_select()

In [44]:
def save_excel(NEW_LIST):
    excel = pd.ExcelWriter('뉴스_IT_20200504.xlsx')
    NEWS_LIST.to_excel(excel,'.',index=False)
    excel.save()

In [45]:
save_excel(NEWS_LIST)